***
### 二次规划模型的一般形式
$$
\begin{align*}
&{\rm max\ (min)}\ \sum\limits_{i=1}^n\sum\limits_{j=1}^nc_{ij}x_ix_j+\sum\limits_{i=1}^nd_ix_i\\
&{\rm s.t.}
	\begin{cases}
	\sum\limits_{j=1}^na_{ij}x_j\leq (\geq,=)\,b_i,\quad i=1,2,\cdots,n,\\
	x_i\geq,\quad i=1,2,\cdots,n,
	\end{cases}\\
&{\text 其中}c_{ij}=c_{ji},\ i,j=1,2,\cdots,n.
\end{align*}
$$

二次规划模型是一种特殊的非线性规划模型，其中
$$
{\boldsymbol H}=
\begin{bmatrix}
c_{11} &c_{12} &\cdots &c_{1n}\\
c_{21} &c_{22} &\cdots &c_{2n}\\
\vdots &\vdots &	   &\vdots\\
c_{n1} &c_{n2} &\cdots &c_{nn}\\
\end{bmatrix}
\in {\mathbb R}^{n\times n}
$$
为对称矩阵。特别地，当 $\boldsymbol H$ 正定时，目标函数最小化时，模型为凸二次规划，凸二次规划局部最优解就是全局最优解

In [1]:
import cvxpy as cp
import numpy as np

c1 =  [98, 277]
c2 = [[-1, -0.15], [-0.15, -2]]
x = cp.Variable(2, nonneg=True)
obj = cp.Maximize(cp.quad_form(x, c2) + c1 @ x)
cons = [
    x[0] + x[1] <= 100, 
    x[0] - 2*x[1] <= 0,
]
prob = cp.Problem(obj, cons)
prob.solve(solver='GUROBI')
print(f'最优解为：{x.value}'), print(f'最优值为：{prob.value}')

最优解为：[35.37037037 64.62962963]
最优值为：11077.870370369614


(None, None)

### 投资组合 (portfolio) 问题

In [2]:
import pandas as pd
df = pd.read_csv('../05第5章  非线性规划和多目标规划模型/data5_4.txt', sep='\t', header=None)
a = df.values
a

array([[ 0.3  ,  0.225,  0.149],
       [ 0.103,  0.29 ,  0.26 ],
       [ 0.216,  0.216,  0.419],
       [-0.056, -0.272, -0.078],
       [-0.071,  0.144,  0.169],
       [ 0.056,  0.107, -0.035],
       [ 0.038,  0.321,  0.133],
       [ 0.089,  0.305,  0.732],
       [ 0.09 ,  0.195,  0.021],
       [ 0.083,  0.39 ,  0.131],
       [ 0.035, -0.072,  0.006],
       [ 0.176,  0.715,  0.908]])

In [3]:
mean = np.mean(a, axis=0)
F = np.cov(a.T)
mean, F

(array([0.08825   , 0.21366667, 0.23458333]),
 array([[0.01106148, 0.01284873, 0.0133593 ],
        [0.01284873, 0.0583917 , 0.05542639],
        [0.0133593 , 0.05542639, 0.09422681]]))

#### Case 1: 希望在年收益率至少达到 $15\%$ 的情况下，将投资组合中股票收益的方差降到最小

In [4]:
expected_return_rate = 0.15
x = cp.Variable(3, nonneg=True)
obj = cp.Minimize(cp.quad_form(x, F))
cons = [
    cp.sum(x) == 1,
    mean @ x >= expected_return_rate,
]
prob = cp.Problem(obj, cons)
prob.solve(solver='GUROBI')
print(f'最优解为：{x.value}'), print(f'最优值为：{prob.value}')

最优解为：[0.52687922 0.35776927 0.11535151]
最优值为：0.022841201324650704


(None, None)

#### Case 2: 希望在方差最大不超过 $0.09$ 的情况下，获得最大的收益

In [5]:
maximum_variance = 0.09
x = cp.Variable(3, nonneg=True)
obj = cp.Maximize(mean @ x)
cons = [
    cp.sum(x) == 1,
    cp.quad_form(x, F) <= maximum_variance
]
prob = cp.Problem(obj, cons)
prob.solve(solver='GUROBI')
print(f'最优解为：{x.value}'), print(f'最优值为：{prob.value}')

最优解为：[1.13801407e-08 5.61775888e-02 9.43822400e-01]
最优值为：0.23340828376866365


(None, None)